In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import datetime
import sys
from tqdm import tqdm
import os
from tqdm import tqdm
import random
import pickle
from glob import glob
import gc
from multiprocessing import Pool, cpu_count

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb, inference_xgb
from utils.metric import compute_comptetition_metric
from utils.feature_contena import Features
from utils.pandas_utils import reduce_mem_usage

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = f"""
execution:
    exp_id: exp_089
    debug: False
    submit: False
    multiprocessing: True

dataset: 
    competition_dir: /kaggle/input/child-mind-institute-detect-sleep-states
    cv_split_path: /kaggle/input/cv_split/train_folds.csv
    train_base_path: /kaggle/input/train_base/train_base.csv
    step_csv_dir: /kaggle/input/save_series_csv/csvs

feature:
    agg_freq: 24 # [step]

seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

CFG["output_dir"] = f"/kaggle/output/{CFG['execution']['exp_id']}"
os.makedirs(CFG["output_dir"], exist_ok=True)

In [3]:
oof = pd.read_parquet(os.path.join("/kaggle/output", os.path.join(CFG["execution"]["exp_id"], "oof.parquet")))
sub = pd.read_csv(os.path.join("/kaggle/output", os.path.join(CFG["execution"]["exp_id"], "submission.csv")))
display(oof.head())
display(sub.head())

,series_id,step,wakeup_oof,onset_oof
0,05e1944c3818,6,-0.004606,0.042293
1,05e1944c3818,18,0.015608,0.037047
2,05e1944c3818,30,0.026090,0.033490
3,05e1944c3818,42,0.028880,0.030517
4,05e1944c3818,54,0.034241,0.025460


,series_id,step,event,score,reduced_score
0,038441c925bb,118494,wakeup,8.320655,8.320655
1,038441c925bb,202302,wakeup,8.218999,8.218999
2,038441c925bb,4974,onset,8.076118,8.076118
3,038441c925bb,159990,onset,7.949413,7.949413
4,038441c925bb,40002,onset,7.940571,7.940571


In [4]:
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
labels.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [7]:
step_per_day = 12 * 60 * 24
WINDOW_SIZE = step_per_day / 2

for sid, oof_df in tqdm(oof.groupby("series_id")):
    sub_df = sub[sub["series_id"] == sid].reset_index(drop=True)
    label_df = labels[labels["series_id"] == sid].sort_values("step").reset_index(drop=True)
    sensor_df = pd.read_csv(os.path.join(CFG["dataset"]["step_csv_dir"], f"{sid}.csv"))

    sleep_time_mean = (label_df["step"] % step_per_day).mean()

    for day in range(100):
        base = day * step_per_day + sleep_time_mean
        sub_window_df = sub_df[(base - WINDOW_SIZE <= sub_df["step"]) & (sub_df["step"] <= base + WINDOW_SIZE)]
        label_window_df = label_df[(base - WINDOW_SIZE <= label_df["step"]) & (label_df["step"] <= base + WINDOW_SIZE)]

        step_min = np.nanmin([sub_window_df["step"].min(), label_window_df["step"].min()])
        step_max = np.nanmax([sub_window_df["step"].max(), label_window_df["step"].max()])
        plot_min = step_min - step_per_day * 0.1
        plot_max = step_max + step_per_day * 0.1

        oof_window_df = oof_df[(plot_min <= oof_df["step"]) & (oof_df["step"] <= plot_max)]
        sub_window_df = sub_df[(plot_min <= sub_df["step"]) & (sub_df["step"] <= plot_max)]
        sensor_window_df = sensor_df[(plot_min <= sensor_df["step"]) & (sensor_df["step"] <= plot_max)]
        label_window_df = label_df[(plot_min <= label_df["step"]) & (label_df["step"] <= plot_max)]
                
        if len(sensor_window_df) == 0:
            continue
        fig, axs = plt.subplots(3, 1, figsize=(10, 7))

        # pred
        axs[0].plot(oof_window_df["step"], oof_window_df["onset_oof"])
        axs[0].plot(oof_window_df["step"], oof_window_df["wakeup_oof"])
        sub_onset_df = sub_window_df[sub_window_df["event"]=="onset"]
        axs[0].scatter(sub_onset_df["step"], sub_onset_df["score"], s=20, c="black", marker="o")
        sub_wakeup_df = sub_window_df[sub_window_df["event"]=="wakeup"]
        axs[0].scatter(sub_wakeup_df["step"], sub_wakeup_df["score"], s=20, c="black", marker="o")
        axs[0].grid()
        axs[0].set_xlim(plot_min, plot_max)
        axs[0].set_ylim(0, 10)
        axs[0].set_ylabel("oof")

        # sensor
        axs[1].plot(sensor_window_df["step"], sensor_window_df["enmo"])
        axs[1].grid()
        axs[1].set_xlim(plot_min, plot_max)
        axs[1].set_ylabel("enmo")
        axs[2].plot(sensor_window_df["step"], sensor_window_df["anglez"])
        axs[2].grid()
        axs[2].set_xlim(plot_min, plot_max)
        axs[2].set_ylabel("anglez")

        # label
        for i in range(3):
            min_step = label_window_df["step"].min()
            max_step = label_window_df["step"].max()
            if min_step == min_step and max_step == max_step:
                axs[i].axvspan(min_step, max_step, color="gray", edgecolor="black", alpha=0.2)
                axs[i].axvline(x=min_step, color="black", lw=1)
                axs[i].axvline(x=max_step, color="black", lw=1)
                axs[i].axvspan(min_step - 12 * 30, min_step + 12 * 30, color="gray", edgecolor="black", alpha=0.1, hatch="/")
                axs[i].axvspan(max_step - 12 * 30, max_step + 12 * 30, color="gray", edgecolor="black", alpha=0.1, hatch="/")
                

        path = f"/kaggle/output/{CFG['execution']['exp_id']}/image/{sid}_{day}.png"
        os.makedirs(os.path.dirname(path), exist_ok=True)
        plt.suptitle(f"sid: {sid}, day: {day}")
        plt.tight_layout()
        plt.savefig(path)
        plt.close()
        plt.show()        

100%|██████████| 277/277 [18:36<00:00,  4.03s/it]
